In [ ]:
import datetime, json, re, time
from selenium import webdriver
import pandas as pd
from sqlitedict import SqliteDict
from collections import defaultdict

#!conda install -y selenium
#!pip install sqlitedict

In [ ]:
def init():
    global driver
    try:
        driver.close()
    except:
        pass
    options = webdriver.ChromeOptions()
    options.add_argument('install-autogenerated-theme=255,0,255')
    driver = webdriver.Chrome('chromedriver', options=options)
    #driver.implicitly_wait(10)

In [ ]:
init()
url = 'https://tableau.alleghenycounty.us/t/PublicSite/views/COVID-19AlleghenyCounty/COVID-19?iframeSizedToWindow=true&:embed=y&:showAppBanner=false&:display_count=no&:showVizHome=no&:origin=viz_share_link'
driver.get(url)

In [ ]:
map_elt = driver.find_element_by_id('tabZoneId3')
map_elt.location

In [ ]:
def get_info(x,y):
    actions = webdriver.common.action_chains.ActionChains(driver)
    actions.move_to_element_with_offset(map_elt, x, y)
    actions.perform()
    try:
        popup = driver.find_element_by_css_selector('.tab-tooltipBR')
    except:
        return None
    offset_x = popup.location['x'] - map_elt.location['x'] - x
    offset_y = popup.location['y'] - map_elt.location['y'] - y
    assert(offset_x == 16 and offset_y == 16)
    info = popup.find_element_by_css_selector('span').text

    # Split on newlines and ignore blank lines
    lines = [line.strip() for line in info.split('\n') if len(line.strip())]
    location_name = lines[0]
    location_info = {x[0].strip():x[1].strip() for x in [line.split(':') for line in lines[1:]]}
    location_info['name'] = location_name
    location_info['x'] = x
    location_info['y'] = y
    return location_info

In [ ]:
locations = {}
pixels = defaultdict(lambda:set())

In [ ]:
map_elt_size = map_elt.size
print(f'map_elt.size: {map_elt_size}')

In [ ]:
for y in range(0, map_elt.size['height'], 2):
    for x in range(0, map_elt.size['width'], 2):
        # Split on newlines and ignore blank lines
        location_info = get_info(x,y)
        if location_info and not location_info['name'] in locations:
            print(location_info)
            locations[location_info['name']] = location_info
        if location_info:
            pixels[location_info['name']].add((x,y))

In [ ]:
achd_covid_table = SqliteDict('achd_covid.db', autocommit=True)

In [ ]:
now_time = datetime.datetime.now()

In [ ]:
achd_covid_table[now_time] = locations

In [ ]:
achd_covid_table['scrape_sessions'] = achd_covid_table['scrape_sessions'] + [now_time]

In [ ]:
data_date = '2020-04-14'
md_elt = {'map_size':map_elt_size,'data_date':data_date}

# TODO: figure out right way to get data date from div, fix '2020-04-??' below
if not 'metadata' in achd_covid_table:
    achd_covid_table['metadata'] = md_elt
else:
    m_dict = achd_covid_table['metadata']
    m_dict[now_time] = md_elt
    achd_covid_table['metadata'] = m_dict  
    

In [ ]:
achd_covid_table['metadata']

In [ ]:
achd_covid_table['scrape_sessions']